In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))  # Resize images to a common size
            images.append(img)
    return images

In [3]:
# Load and preprocess data
foul_images = load_images_from_folder('/kaggle/input/op-dataset/OP_Dataset/Train/foul')
non_foul_images = load_images_from_folder('/kaggle/input/op-dataset/OP_Dataset/Train/non_foul')

# Create labels (1 for foul, 0 for non-foul)
foul_labels = np.ones(len(foul_images))
non_foul_labels = np.zeros(len(non_foul_images))

# Combine data and labels
data = np.array(foul_images + non_foul_images)
labels = np.concatenate((foul_labels, non_foul_labels))

val_foul_images = load_images_from_folder("/kaggle/input/op-dataset/OP_Dataset/Valid/foul")
val_non_foul_images = load_images_from_folder("/kaggle/input/op-dataset/OP_Dataset/Valid/non_foul")

val_foul_labels = np.ones(len(val_foul_images))
val_non_foul_labels = np.zeros(len(val_non_foul_images))

val_data = np.array(val_foul_images + val_non_foul_images)
val_labels = np.concatenate((val_foul_labels, val_non_foul_labels))


# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,  # Increased rotation range
    width_shift_range=0.3,  # Increased width shift range
    height_shift_range=0.3,  # Increased height shift range
    shear_range=0.3,  # Increased shear range
    zoom_range=0.3,  # Increased zoom range
    horizontal_flip=True,
    fill_mode='nearest'
)

# Assuming your labels include both classification labels and bounding box coordinates
train_generator = datagen.flow(X_train, y_train, batch_size=32)


input_shape = (224, 224, 3)
num_classes = 1  # Binary classification for foul detection

# Build the EfficientNetB0 model
base_model = tf.keras.applications.EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Feature extraction
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

# Classification head
classification_head = tf.keras.layers.Dense(num_classes, activation='sigmoid', name='classification')(x)

# Build the model
model = tf.keras.Model(inputs=base_model.input, outputs=classification_head)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Learning Rate Scheduler
def lr_scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch > 0:
        lr = lr * 0.9
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

# Train the model
history = model.fit(train_generator,
                    epochs=100,
                    validation_data=(X_val, y_val),
                    callbacks=[lr_schedule])

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print("Validation Accuracy:", val_accuracy)

16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/100


2024-01-12 11:19:49.659860: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


31/31 [==============================] - 58s 509ms/step - loss: 0.7145 - accuracy: 0.5254 - val_loss: 0.6581 - val_accuracy: 0.6266 - lr: 1.0000e-04
Epoch 2/100
31/31 [==============================] - 13s 426ms/step - loss: 0.6900 - accuracy: 0.5576 - val_loss: 0.6436 - val_accuracy: 0.6763 - lr: 1.0000e-04
Epoch 3/100
31/31 [==============================] - 13s 422ms/step - loss: 0.6685 - accuracy: 0.5826 - val_loss: 0.6211 - val_accuracy: 0.6929 - lr: 1.0000e-04
Epoch 4/100
31/31 [==============================] - 13s 410ms/step - loss: 0.6370 - accuracy: 0.6449 - val_loss: 0.5968 - val_accuracy: 0.7303 - lr: 1.0000e-04
Epoch 5/100
31/31 [==============================] - 13s 423ms/step - loss: 0.5960 - accuracy: 0.6791 - val_loss: 0.5616 - val_accuracy: 0.7344 - lr: 1.0000e-04
Epoch 6/100
31/31 [==============================] - 13s 418ms/step - loss: 0.5615 - accuracy: 0.7155 - val_loss: 0.5349 - val_accuracy: 0.7386 - lr: 1.0000e-04
Epoch 7/100
31/31 [===========================

In [6]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(val_data)
conf_matrix = confusion_matrix(val_labels, predictions > 0.5)
print("Confusion Matrix:")
print(conf_matrix)

15/15 [==============================] - 1s 58ms/step
Confusion Matrix:
[[247   2]
 [  3 201]]


In [7]:
model.save("foul_detection_model.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0][0]']   

In [9]:
def detect_foul(image_path, foul_model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize image to match your model's input size
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = foul_model.predict(img)
    if prediction >= 0.5:
        return "foul"
    else:
        return "not-foul"

In [10]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/football-tackles/var500/VAR/Clean_Tackles/109.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 2s 2s/step
Detection Result: not-foul


In [11]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/football-tackles/var500/VAR/Fouls/10.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 2s 2s/step
Detection Result: foul


In [12]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/football-tackles/var500/VAR/Fouls/108.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 2s 2s/step
Detection Result: foul


In [12]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/football-tackles/var500/VAR/Clean_Tackles/111.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 2s 2s/step
Detection Result: not-foul


In [13]:
from tensorflow.keras.models import load_model
def process_folder(folder_path, foul_model):
    foul_counter = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more extensions if needed
            image_path = os.path.join(folder_path, filename)
            result = detect_foul(image_path, foul_model)

            if result == "foul":
                foul_counter += 1

    return foul_counter

# Example usage:
foul_model_path = '/kaggle/working/foul_detection_model.h5'
foul_model = load_model(foul_model_path)
folder_path = "/kaggle/input/op-dataset/OP_Dataset/Test/foul"
total_fouls = process_folder(folder_path, foul_model)

print(f"Total fouls detected: {total_fouls}")

1/1 [==============================] - 0s 25ms/step
Total fouls detected: 394


In [14]:
from tensorflow.keras.models import load_model
def process_folder(folder_path, foul_model):
    foul_counter = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more extensions if needed
            image_path = os.path.join(folder_path, filename)
            result = detect_foul(image_path, foul_model)

            if result == "foul":
                foul_counter += 1

    return foul_counter

# Example usage:
foul_model_path = '/kaggle/working/foul_detection_model.h5'
foul_model = load_model(foul_model_path)
folder_path = "/kaggle/input/op-dataset/OP_Dataset/Test/non_foul"
total_fouls = process_folder(folder_path, foul_model)

print(f"Total fouls detected: {total_fouls}")

1/1 [==============================] - 0s 23ms/step
Total fouls detected: 7


In [ ]:
import wget
import os

def download_model(model_url, save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    model_filename = os.path.join(save_path, 'foul_detection_model.h5')

    if not os.path.exists(model_filename):
        print(f"Downloading model from {model_url}")
        wget.download(model_url, out=model_filename)
        print("\nDownload complete.")
    else:
        print("Model already exists.")

    return model_filename

# Example usage:
model_url = 'https://example.com/path/to/foul_detection_model.h5'
save_path = '/path/to/save'

downloaded_model_path = download_model(model_url, save_path)

In [32]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/soccerdata/dataset/test/foul/209.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 2s 2s/step
Detection Result: not-foul


In [17]:
model_path = '/kaggle/working/foul_detection_model.h5'
foul_detection_model = tf.keras.models.load_model(model_path)
image_path = '/kaggle/input/soccerdata/dataset/test/nonfoul/111.jpg'
result = detect_foul(image_path, foul_detection_model)
print("Detection Result:", result)

1/1 [==============================] - 2s 2s/step
Detection Result: not-foul


In [16]:
from tensorflow.keras.models import load_model
def process_folder(folder_path, foul_model):
    foul_counter = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more extensions if needed
            image_path = os.path.join(folder_path, filename)
            result = detect_foul(image_path, foul_model)

            if result == "foul":
                foul_counter += 1

    return foul_counter

# Example usage:
foul_model_path = '/kaggle/working/foul_detection_model.h5'
foul_model = load_model(foul_model_path)
folder_path = "/kaggle/input/videoframes/kaggle/working/image"
total_fouls = process_folder(folder_path, foul_model)

print(f"Total fouls detected: {total_fouls}")

1/1 [==============================] - 0s 23ms/step
Total fouls detected: 309
